In [2]:
import tensorflow as tf

# tf.keras.applications.resnet50.preprocess_input

2025-08-24 21:37:23.227729: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-24 21:37:23.266108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-24 21:37:23.996810: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [1]:
!wget https://www.mygoldenretrieverpuppies.com/wp-content/uploads/2022/06/Golden-Retriever-Puppies.jpeg

--2025-08-24 21:37:19--  https://www.mygoldenretrieverpuppies.com/wp-content/uploads/2022/06/Golden-Retriever-Puppies.jpeg
Resolving www.mygoldenretrieverpuppies.com (www.mygoldenretrieverpuppies.com)... 141.193.213.10, 141.193.213.11
Connecting to www.mygoldenretrieverpuppies.com (www.mygoldenretrieverpuppies.com)|141.193.213.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154638 (151K) [image/jpeg]
Saving to: ‘Golden-Retriever-Puppies.jpeg’

Golden-Retriever-Pu 100%[===================>] 151.01K  --.-KB/s    in 0.03s   

2025-08-24 21:37:19 (4.44 MB/s) - ‘Golden-Retriever-Puppies.jpeg’ saved [154638/154638]



In [5]:
!wget https://image.jimcdn.com/app/cms/image/transf/none/path/scb57eb606df3e140/backgroundarea/ib321d747619369f9/version/1518116761/image.jpg

--2025-08-24 21:38:44--  https://image.jimcdn.com/app/cms/image/transf/none/path/scb57eb606df3e140/backgroundarea/ib321d747619369f9/version/1518116761/image.jpg
Resolving image.jimcdn.com (image.jimcdn.com)... 2606:4700:4400::ac40:9a45, 2606:4700:4400::6812:21bb, 172.64.154.69, ...
Connecting to image.jimcdn.com (image.jimcdn.com)|2606:4700:4400::ac40:9a45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810378 (791K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>] 791.38K  --.-KB/s    in 0.1s    

2025-08-24 21:38:44 (6.28 MB/s) - ‘image.jpg’ saved [810378/810378]



In [7]:
# Load the model from the checkpoint
model = tf.keras.models.load_model(
    "/home/ubuntu/dev/carni_detect/models/carnidetectv1.keras"
)

image = tf.keras.preprocessing.image.load_img(
    "/home/ubuntu/dev/carni_detect/notebooks/image.jpg",
    target_size=(224, 224),
)
image = tf.keras.preprocessing.image.img_to_array(image)
image = tf.expand_dims(image, axis=0)
image = tf.keras.applications.resnet50.preprocess_input(image)

# Make predictions
predictions = model.predict(image)
print("Predictions:", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predictions: [[0.    0.    0.    0.008 0.013 0.047 0.    0.002 0.    0.    0.153 0.005
  0.755 0.015 0.001]]


In [22]:
def preprocess_dataset(dataset: tf.data.Dataset):
    dataset = dataset.map(
        lambda image, label: (
            tf.keras.applications.resnet50.preprocess_input(image),
            label,
        )
    )
    return dataset


def get_image_dataset(
    data_directory: str, batch_size: int, shuffle: bool
) -> tf.data.Dataset:
    """Creates a TensorFlow dataset from the images in the specified directory.

    Args:
        data_directory (str): Path to the directory containing the image files.
        batch_size (int): Number of images to include in each batch.

    Returns:
        tf.data.Dataset: A TensorFlow dataset object.
    """
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_directory,
        label_mode="categorical",
        seed=42,
        batch_size=batch_size,
        image_size=(224, 224),  # Resize images to a consistent size
        shuffle=shuffle,
        color_mode="rgb",
    )
    dataset = preprocess_dataset(dataset)
    return dataset


test_dataset = get_image_dataset(
    data_directory="/home/ubuntu/dev/carni_detect/dataset/test",
    batch_size=32,
    shuffle=False,
)

Found 375 files belonging to 15 classes.


In [23]:
pred = model.evaluate(test_dataset)
print(pred)

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - categorical_accuracy: 0.9493 - loss: 0.1795
[0.17947760224342346, 0.9493333101272583]


In [4]:
import numpy as np

np.set_printoptions(precision=3, suppress=True)
print("Predictions:", predictions)

Predictions: [[0.    0.    0.    0.    0.    0.    0.    0.    0.999 0.    0.001 0.
  0.    0.    0.   ]]


In [18]:
final_pred = tf.argmax(predictions, axis=1)
print(final_pred)

tf.Tensor([8], shape=(1,), dtype=int64)
